In [1]:
from collections import namedtuple
import re
import pdfplumber
import pandas as pd

In [2]:
Line = namedtuple('Line', 'fund_name date description gross_amt deductions net_amt unit_price units units_held')

In [3]:
file = 'sample report.pdf'

In [57]:
fund_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (.*) (.*) (.*) (.*) (.*) (.*) (.*)')
closing_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (Closing Balance) (.*) (.*) (.*)')
cash_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (Distribution\s......) (.*) (.*) (.*) (.*)')

In [58]:
## Gut-check for pdf file
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        print(text)


 
 
 
 
 
T +61 3 9098 8088 F +61 3 9098 8099
info@primevalue.com.au  www.primevalue.com.au
Mr Andrew Russell Issue Date: 13/01/2022
8 Wolseley Crescent Period Start Date: 04/05/2021
Point Piper NSW Period End Date: 31/12/2021
Account No: HNW11223
Transaction History Report
Prime Value SIV Enhanced Income Fund
Date Description Gross Amount Deductions Net Amount Unit Price Units Units Held
04/05/2021 Initial Application $1,000,000.00 $0.00 $1,000,000.00 0.9979 1,002,104.42 1,002,104.42
30/06/2021 Distribution Reinvestment $6,112.84 $572.59 $0.00 0.9967 0.00 1,002,104.42
30/09/2021 Distribution Reinvestment $4,008.42 $292.31 $3,716.11 0.9948 3,735.53 1,005,839.95
31/12/2021 Distribution Reinvestment $4,023.36 $0.00 $4,023.36 0.9931 4,051.31 1,009,891.26
31/12/2021 Closing Balance $1,006,760.60 0.9969 1,009,891.26
Prime Value SIV Opportunities Fund
Date Description Gross Amount Deductions Net Amount Unit Price Units Units Held
04/05/2021 Initial Application $1,000,000.00 $0.00 $1,000,000.

In [97]:
## Check if re.search works
for line in text.split('\n'):
    if fund_line_re.search(line):
        print(line)
    if cash_line_re.search(line):
        print(line)
    if closing_line_re.search(line):
        print(line)

04/05/2021 Initial Application $1,500,000.00 $0.00 $1,500,000.00 1.8411 814,730.32 814,730.32
31/12/2021 Closing Balance $1,597,034.37 1.9602 814,730.32


In [94]:
##Original
data = []
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        for line in text.split('\n'):
            if fund_re.match(line):
                fund_name = line.split()
                fund_name = ' '.join(fund_name)

            fund_line = fund_line_re.search(line)
            closing= closing_line_re.search(line)
            cash= cash_line_re.search(line)
            if fund_line:
                date = fund_line.group(1)
                description = fund_line.group(2)
                gross_amt = fund_line.group(3).replace('$', '').replace(',', '')
                deductions = fund_line.group(4).replace('$', '').replace(',', '')
                net_amt = fund_line.group(5).replace('$', '').replace(',', '')
                unit_price = fund_line.group(6)
                units = fund_line.group(7)
                units_held = fund_line.group(8).replace('$', '').replace(',', '')
                data.append(Line(fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
            elif closing:
                date = closing.group(1)
                description = closing.group(2)
                gross_amt = '0.00'
                deductions = '0.00'
                net_amt = closing.group(3).replace('$', '').replace(',', '')
                unit_price = closing.group(4)
                units_held = closing.group(5).replace('$', '').replace(',', '')
                data.append(Line(fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
            elif cash:
                date = cash.group(1)
                description = cash.group(2)
                gross_amt = cash.group(3).replace('$', '').replace(',', '')
                deductions = cash.group(4).replace('$', '').replace(',', '')
                net_amt = cash.group(5).replace('$', '').replace(',', '')
                unit_price = '0.0000'
                units = '0.00'
                units_held = cash.group(6).replace('$', '').replace(',', '')
                data.append(Line(fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
                

In [95]:
df = pd.DataFrame(data)

In [96]:
df.head(40)

,fund_name,date,description,gross_amt,deductions,net_amt,unit_price,units,units_held
0,Prime Value SIV Enhanced Income Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,0.9979,"1,002,104.42",1002104.42
1,Prime Value SIV Enhanced Income Fund,30/06/2021,Distribution Reinvestment,6112.84,572.59,0.00,0.9967,0.00,1002104.42
2,Prime Value SIV Enhanced Income Fund,30/09/2021,Distribution Reinvestment,4008.42,292.31,3716.11,0.9948,"3,735.53",1005839.95
3,Prime Value SIV Enhanced Income Fund,31/12/2021,Distribution Reinvestment,4023.36,0.00,4023.36,0.9931,"4,051.31",1009891.26
4,Prime Value SIV Enhanced Income Fund,31/12/2021,Closing Balance,0.00,0.00,1006760.60,0.9969,"4,051.31",1009891.26
5,Prime Value SIV Opportunities Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,1.8667,"535,704.72",535704.72
6,Prime Value SIV Opportunities Fund,30/06/2021,Distribution (Cash),32356.57,112.06,17219.49,0.0000,0.00,535704.72
7,Prime Value SIV Opportunities Fund,31/12/2021,Closing Balance,0.00,0.00,1068034.50,1.9937,0.00,535704.72
8,Prime Value SIV Growth Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,1.8578,"538,271.07",538271.07
9,Prime Value SIV Growth Fund,30/06/2021,Distribution Reinvestment,21154.05,98.26,18336.82,1.9108,"9,596.41",547867.48


In [70]:
df[date] =pd.to_datetime(df[date])

KeyError: '31/12/2021'

In [ ]:
df.to_csv('final.csv', index=False)